In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords

stops = set(stopwords.words('english'))

import numpy as np
from sklearn.feature_extraction.text import CountVectorizer as CV
import string
exclude = set(string.punctuation)

def process_labels(x):
    t = str(x).lower().strip('"').strip('[').strip(' ').strip('\n').strip(' ')
    if t=='misinformation':
        return 1
    elif t=='not misinformation':
        return 0
    elif ('not' in t) or ('mis' in t):
        return 0
    else:
        return np.nan

In [2]:
df = pd.read_csv('data/fightin_words.csv')

df['label_gpt3.5'] = df['label_gpt3.5'].apply(lambda x: process_labels(x))  
df['label_gpt4'] = df['label_gpt4'].apply(lambda x: process_labels(x))

In [3]:
def basic_sanitize(in_string):
    '''Returns a very roughly sanitized version of the input string.'''  
    in_string = ''.join([ch for ch in in_string if ch not in exclude])
    in_string = in_string.lower()
    in_string = ' '.join(in_string.split())
    return in_string

def bayes_compare_language(l1, l2, ngram = 1, prior=.01, cv = None):
    '''
    Arguments:
    - l1, l2; a list of strings from each language sample
    - ngram; an int describing up to what n gram you want to consider (1 is unigrams,
    2 is bigrams + unigrams, etc). Ignored if a custom CountVectorizer is passed.
    - prior; either a float describing a uniform prior, or a vector describing a prior
    over vocabulary items. If you're using a predefined vocabulary, make sure to specify that
    when you make your CountVectorizer object.
    - cv; a sklearn.feature_extraction.text.CountVectorizer object, if desired.
    Returns:
    - A list of length |Vocab| where each entry is a (n-gram, zscore) tuple.'''
    if cv is None and type(prior) is not float:
        print("If using a non-uniform prior:")
        print("Please also pass a count vectorizer with the vocabulary parameter set.")
        quit()
    l1 = [basic_sanitize(l) for l in l1]
    l2 = [basic_sanitize(l) for l in l2]
    if cv is None:
        cv = CV(decode_error = 'ignore', min_df = 3, max_df = .5, stop_words = list(stops) + ['timeslive','amp','sundaytimesza','headline','cj','littlefamilyoz','benmitchellsong','africaupdates','davidfrum','africa','south','devil','god','ur','world'],
                ngram_range=(1,ngram),
                binary = False,
                max_features = 15000)
    counts_mat = cv.fit_transform(l1+l2).toarray()
    # Now sum over languages...
    vocab_size = len(cv.vocabulary_)
    print("Vocab size is {}".format(vocab_size))
    if type(prior) is float:
        priors = np.array([prior for i in range(vocab_size)])
    else:
        priors = prior
    z_scores = np.empty(priors.shape[0])
    count_matrix = np.empty([2, vocab_size], dtype=np.float32)
    count_matrix[0, :] = np.sum(counts_mat[:len(l1), :], axis = 0)
    count_matrix[1, :] = np.sum(counts_mat[len(l1):, :], axis = 0)
    a0 = np.sum(priors)
    n1 = 1.*np.sum(count_matrix[0,:])
    n2 = 1.*np.sum(count_matrix[1,:])
    print("Comparing language...")
    for i in range(vocab_size):
        #compute delta
        term1 = np.log((count_matrix[0,i] + priors[i])/(n1 + a0 - count_matrix[0,i] - priors[i]))
        term2 = np.log((count_matrix[1,i] + priors[i])/(n2 + a0 - count_matrix[1,i] - priors[i]))        
        delta = term1 - term2
        #compute variance on delta
        var = 1./(count_matrix[0,i] + priors[i]) + 1./(count_matrix[1,i] + priors[i])
        #store final score
        z_scores[i] = delta/np.sqrt(var)
    index_to_term = {v:k for k,v in cv.vocabulary_.items()}
    sorted_indices = np.argsort(z_scores)
    return_list = []
    for i in sorted_indices:
        return_list.append((index_to_term[i], z_scores[i]))
    return return_list

### gpt4

In [4]:
df_pos = (df.loc[((df['label_gpt4']==1)) & (df['type']=='disapproving')]).reset_index()['text']
df_neg = (df.loc[((df['label_gpt4']==0)) & (df['type']=='disapproving')]).reset_index()['text']

In [5]:
l1 = list(df_pos)
l2 = list(df_neg)

res = bayes_compare_language((l1), (l2),  ngram = 1, prior= 0.01,
                             cv = None)
res = pd.DataFrame(res)
res['word'] = res[0]
res['z-score'] = res[1]

#keep only significant (95% CI)

#res = res.loc[(res['z-score'] < -1.97) |  (res['z-score'] > 1.97)]

Vocab size is 959
Comparing language...


In [6]:
words = res[['word', 'z-score']].loc[res['z-score'] <0].sort_values(by= 'z-score', ascending = True)['word'].values
scores = res[['word', 'z-score']].loc[res['z-score'] <0].sort_values(by= 'z-score', ascending = True)['z-score'].values

print(", ".join("%s (%.2f)"%(words[i],scores[i]) for i in range(len(words))[:15]))

please (-3.67), vaccine (-3.61), therapy (-3.53), gene (-3.32), misinformation (-3.23), stop (-2.82), mrna (-2.52), dna (-2.43), uses (-2.41), wrong (-2.27), change (-2.10), correct (-2.03), safe (-2.01), cdc (-1.99), well (-1.93)


In [7]:
words = res[['word', 'z-score']].loc[res['z-score'] >0].sort_values(by= 'z-score', ascending = True)['word'].values[::-1]
scores = res[['word', 'z-score']].loc[res['z-score'] >0].sort_values(by= 'z-score', ascending = True)['z-score'].values[::-1]

print(", ".join("%s (%.2f)"%(words[i],scores[i]) for i in range(len(words))[:15]))

fake (5.03), news (4.89), lying (4.27), lies (4.16), one (3.94), misleading (3.84), lie (3.76), put (3.43), thing (3.35), always (3.13), false (3.10), new (2.98), everything (2.83), anyway (2.83), country (2.83)


### gpt3.5

In [8]:
df_pos = (df.loc[((df['label_gpt3.5']==1)) & (df['type']=='disapproving')]).reset_index()['text']
df_neg = (df.loc[((df['label_gpt3.5']==0)) & (df['type']=='disapproving')]).reset_index()['text']

In [9]:
l1 = list(df_pos)
l2 = list(df_neg)

res = bayes_compare_language((l1), (l2),  ngram = 1, prior= 0.01,
                             cv = None)
res = pd.DataFrame(res)
res['word'] = res[0]
res['z-score'] = res[1]

Vocab size is 948
Comparing language...


In [10]:
words = res[['word', 'z-score']].loc[res['z-score'] <0].sort_values(by= 'z-score', ascending = True)['word'].values
scores = res[['word', 'z-score']].loc[res['z-score'] <0].sort_values(by= 'z-score', ascending = True)['z-score'].values

print(", ".join("%s (%.2f)"%(words[i],scores[i]) for i in range(len(words))[:15]))

mrna (-4.24), vaccine (-3.40), dna (-2.82), uses (-2.82), change (-2.77), please (-2.17), genes (-2.15), genetic (-2.12), actually (-2.09), gene (-2.06), therapy (-2.06), understand (-2.02), correct (-1.86), immunization (-1.86), nothing (-1.79)


In [11]:
words = res[['word', 'z-score']].loc[res['z-score'] >0].sort_values(by= 'z-score', ascending = True)['word'].values[::-1]
scores = res[['word', 'z-score']].loc[res['z-score'] >0].sort_values(by= 'z-score', ascending = True)['z-score'].values[::-1]

print(", ".join("%s (%.2f)"%(words[i],scores[i]) for i in range(len(words))[:15]))

fake (5.39), misleading (5.08), lying (4.31), lies (4.20), media (4.09), news (4.03), another (3.60), report (3.39), journalism (3.37), blood (2.94), false (2.92), crazy (2.65), justice (2.64), remove (2.48), reporting (2.45)
